**Computational Social Science**

*Assignment 1*

Link to github: https://github.com/Dribo/02467_A_1

*Contribution statement*

All the exercises have been completed in cooperation, where we have worked to complete one assignment before moving on to the next.

**Part 1: Using web-scraping to gather data**
The following cells contain the code for solve part 1 of the exercise

In [19]:
# Imports
from bs4 import BeautifulSoup
import requests
import re

In [20]:
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

# Getting the web-page
link19oral = "https://2019.ic2s2.org/oral-presentations/"
link19post = "https://2019.ic2s2.org/posters/"
r = requests.get(link19oral, verify=False)
soup = BeautifulSoup(r.content)
div = soup.find("div", {"class":"col-md-8"})
ps = div.findAll('p')
relevant = ps[3:]

G:\anaconda3\envs\02467 - css\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '2019.ic2s2.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [21]:
# Creating regex for filtering out names on the web-page for oral presentations
# regex1 is for names that have an initial in the middle, regex2 is for all other names
regex1 = r" ([a-zA-ZÀ-ÿ]* ([A-Z]. )+[a-zA-ZÀ-ÿ]+[.,])"
regex2 = r" ([a-zA-ZÀ-ÿ]* [a-zA-ZÀ-ÿ]+[.,])"
names = []
names1 = []
# Here we loop through the relevant part of the web-page and use the RegEx to find all the names and add them to a list
for r in relevant:
    retrieved = re.findall(regex1, str(r), re.UNICODE)
    names += retrieved

# Remove trailing spaces/dots
for i in range(len(names)):
    names[i] = names[i][0][:-1]

# This is the same but for regex2
for r in relevant:
    retrieved = re.findall(regex2, str(r), re.UNICODE)
    names1 += retrieved

for i in range(len(names1)):
    names1[i] = names1[i][:-1]

names += names1
list_of_presenters = names

In [22]:
# Getting the web-page for the posters
r = requests.get(link19post, verify=False)
soup = BeautifulSoup(r.content)
uls = soup.findAll('ul')
r1 = list(uls)[6]
r2 = list(uls)[7]

relevant1 = []
relevant2 = []

# Splitting the strings at new-lines
for x in list(r1):
    if x != '\n':
        relevant1.append(x)
for x in list(r2):
    if x != '\n':
        relevant2.append(x)

G:\anaconda3\envs\02467 - css\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '2019.ic2s2.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# Regex for filtering of posters
regex1 = r"<li>[a-zA-Z ]*<span>"
names = []
names1 = []
names2 = []
names3 = []
# The same as before where we use the regex to filter the strings and find the names
for r in relevant1:
    retrieved = re.findall(regex1, str(r), re.UNICODE)
    names += retrieved

for i in range(len(names)):
    names[i] = names[i][4:-6]
    names[i] = names[i].split(' and ')
    for j in range(len(names[i])):
        names1 += [names[i].pop(0)]

for r in relevant2:
    retrieved = re.findall(regex1, str(r), re.UNICODE)
    names2 += retrieved

for i in range(len(names2)):
    names2[i] = names2[i][4:-6]
    names2[i] = names2[i].split(' and ')
    for j in range(len(names2[i])):
        names3 += [names2[i].pop(0)]

list_of_presenters += names1 + names3
# Create the list of presenters and turn it into a set to remove dublicates
print('List of presenters with dublicates:', len(list_of_presenters))
print('List of presnters without dublicates :',  len(set(list_of_presenters)))

List of presenters with dublicates: 742
List of presnters without dublicates : 613


*Part 1*

    2. We got 613 unique authors in 2019 for both posters and oral presentations

    3. One decision that was taken during the web-scraping was to use RegEx to filter the out everything but the names. What this did was result in some oddities, where some elements got included that weren't names, and some names were cut off a bit too early, especially if they contained an intial as a middle name.